**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# NGinx unprivileged

We are using the [vendors original recipe](https://hub.docker.com/r/nginxinc/nginx-unprivileged), should be save ☠️



In [1]:
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-strict
  labels:
    app: nginx-strict
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nginx-strict
  template:
    metadata:
      labels:
        app: nginx-strict
    spec:
      containers:
      - name: nginx
        image: nginxinc/nginx-unprivileged
        ports:
        - containerPort: 8080
EOF

deployment.apps/nginx-strict created


In [5]:
kubectl get pods

NAME                            READY   STATUS    RESTARTS   AGE
my-nginx-5b56ccd65f-4xxg2       1/1     Running   0          4h4m
my-nginx-5b56ccd65f-mhqpf       1/1     Running   0          4h46m
nginx-strict-5cdc48cf55-jmjhj   1/1     Running   0          20s
nginx-strict-5cdc48cf55-rr4kc   1/1     Running   0          20s
nginx-strict-5cdc48cf55-vz7ds   1/1     Running   0          20s


In [3]:
kubectl apply -f - <<EOF
apiVersion: v1
kind: Service
metadata:
  name: nginx-strict
  namespace: default
spec:
  ports:
  - port: 80
    protocol: TCP
    targetPort: 8080
  selector:
    app: nginx-strict
  sessionAffinity: None
  type: LoadBalancer
EOF

service/nginx-strict created


In [6]:
kubectl get svc

NAME           TYPE           CLUSTER-IP       EXTERNAL-IP      PORT(S)        AGE
kubernetes     ClusterIP      10.96.0.1        <none>           443/TCP        6h3m
my-nginx       ClusterIP      10.109.82.109    <none>           80/TCP         4h47m
nginx-strict   LoadBalancer   10.102.198.117   10.102.198.117   80:30664/TCP   64s


In [7]:
kubectl get svc nginx-strict  

NAME           TYPE           CLUSTER-IP       EXTERNAL-IP      PORT(S)        AGE
nginx-strict   LoadBalancer   10.102.198.117   10.102.198.117   80:30664/TCP   84s


In [8]:
curl 10.102.198.117

<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
html { color-scheme: light dark; }
body { width: 35em; margin: 0 auto;
font-family: Tahoma, Verdana, Arial, sans-serif; }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>


In [ ]:
kubectl get svc nginx-strict -o yaml

In [9]:
kubectl delete deployment my-nginx

deployment.apps "my-nginx" deleted


In [10]:
kubectl delete service my-nginx

service "my-nginx" deleted


In [11]:
kubectl get pods -o jsonpath='{.items[0].metadata.name}' 

nginx-strict-5cdc48cf55-jmjhj

In [12]:
POD=$(kubectl get pods -o jsonpath='{.items[0].metadata.name}' | grep nginx-strict)
echo $POD

nginx-strict-5cdc48cf55-jmjhj


In [13]:
kubectl exec $POD --  curl -L -o /tmp/kubectl https://dl.k8s.io/release/v1.20.0/bin/linux/amd64/kubectl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   154  100   154    0     0    536      0 --:--:-- --:--:-- --:--:--   534
100 38.3M  100 38.3M    0     0  29.0M      0  0:00:01  0:00:01 --:--:-- 65.1M


In [15]:
kubectl exec $POD -- ls /tmp

client_temp
fastcgi_temp
kubectl
nginx.pid
proxy_temp
scgi_temp
uwsgi_temp


In [16]:
kubectl exec $POD -- chmod +x /tmp/kubectl

In [17]:
kubectl exec $POD -- /tmp/kubectl

kubectl controls the Kubernetes cluster manager.

 Find more information at: https://kubernetes.io/docs/reference/kubectl/overview/

Basic Commands (Beginner):
  create        Create a resource from a file or from stdin.
  expose        Take a replication controller, service, deployment or pod and expose it as a new Kubernetes Service
  run           Run a particular image on the cluster
  set           Set specific features on objects

Basic Commands (Intermediate):
  explain       Documentation of resources
  get           Display one or many resources
  edit          Edit a resource on the server
  delete        Delete resources by filenames, stdin, resources and names, or by resources and label selector

Deploy Commands:
  rollout       Manage the rollout of a resource
  scale         Set a new size for a Deployment, ReplicaSet or Replication Controller
  autoscale     Auto-scale a Deployment, ReplicaSet, or ReplicationController

Cluster Management Commands:
  certificate   Modify

In [18]:
kubectl exec $POD -- /tmp/kubectl get pods 

Error from server (Forbidden): pods is forbidden: User "system:serviceaccount:default:default" cannot list resource "pods" in API group "" in the namespace "default"
command terminated with exit code 1


: 1

In [19]:
kubectl exec $POD  -- df

Filesystem             1K-blocks      Used Available Use% Mounted on
overlay                308587328 231971864  60870440  80% /
tmpfs                      65536         0     65536   0% /dev
tmpfs                   32632360         0  32632360   0% /sys/fs/cgroup
/dev/mapper/system-var 308587328 231971864  60870440  80% /etc/hosts
shm                        65536         0     65536   0% /dev/shm
tmpfs                   65264724        12  65264712   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs                   32632360         0  32632360   0% /proc/acpi
tmpfs                   32632360         0  32632360   0% /proc/scsi
tmpfs                   32632360         0  32632360   0% /sys/firmware


In [20]:
kubectl get serviceaccounts -A

NAMESPACE              NAME                                 SECRETS   AGE
default                default                              1         6h5m
kube-node-lease        default                              1         6h5m
kube-public            default                              1         6h5m
kube-system            attachdetach-controller              1         6h5m
kube-system            bootstrap-signer                     1         6h5m
kube-system            certificate-controller               1         6h5m
kube-system            clusterrole-aggregation-controller   1         6h5m
kube-system            coredns                              1         6h5m
kube-system            cronjob-controller                   1         6h5m
kube-system            daemon-set-controller                1         6h5m
kube-system            default                              1         6h5m
kube-system            deployment-controller                1         6h5m
kube-system            dis

#  💀 💀 💀 💀 💀 💀 💀 💀 💀
# 💀 💀 **Never Ever** 💀 💀 💀 
#  💀 💀 💀 💀 💀 💀 💀 💀 💀


In [21]:
kubectl create clusterrolebinding cluster-admin-binding --clusterrole cluster-admin --serviceaccount default:default

clusterrolebinding.rbac.authorization.k8s.io/cluster-admin-binding created


In [22]:
kubectl exec $POD -- /tmp/kubectl get pods -A

NAMESPACE              NAME                                         READY   STATUS    RESTARTS       AGE
default                nginx-strict-5cdc48cf55-jmjhj                1/1     Running   0              4m4s
default                nginx-strict-5cdc48cf55-rr4kc                1/1     Running   0              4m4s
default                nginx-strict-5cdc48cf55-vz7ds                1/1     Running   0              4m4s
kube-system            coredns-bcc9dbffc-nxcjt                      1/1     Running   0              143m
kube-system            etcd-minikube                                1/1     Running   0              6h6m
kube-system            kube-apiserver-minikube                      1/1     Running   0              6h6m
kube-system            kube-controller-manager-minikube             1/1     Running   0              6h6m
kube-system            kube-proxy-bffmk                             1/1     Running   0              6h5m
kube-system            kube-scheduler-minikube 

In [23]:
kubectl exec $POD -- /tmp/kubectl run malicious --image=nginx

pod/malicious created


In [24]:
kubectl get pod

NAME                            READY   STATUS    RESTARTS   AGE
malicious                       1/1     Running   0          4s
nginx-strict-5cdc48cf55-jmjhj   1/1     Running   0          4m45s
nginx-strict-5cdc48cf55-rr4kc   1/1     Running   0          4m45s
nginx-strict-5cdc48cf55-vz7ds   1/1     Running   0          4m45s


In [25]:
kubectl exec $POD -- /tmp/kubectl delete pod malicious 

pod "malicious" deleted


In [26]:
kubectl get clusterroles

NAME                                                                   CREATED AT
admin                                                                  2022-01-18T09:25:25Z
cluster-admin                                                          2022-01-18T09:25:25Z
edit                                                                   2022-01-18T09:25:25Z
kubeadm:get-nodes                                                      2022-01-18T09:25:27Z
kubernetes-dashboard                                                   2022-01-18T12:56:14Z
system:aggregate-to-admin                                              2022-01-18T09:25:25Z
system:aggregate-to-edit                                               2022-01-18T09:25:25Z
system:aggregate-to-view                                               2022-01-18T09:25:25Z
system:auth-delegator                                                  2022-01-18T09:25:25Z
system:basic-user                                                      2022-01-18T09:25:25

In [27]:
kubectl get clusterrolebindings

NAME                                                   ROLE                                                                               AGE
cluster-admin                                          ClusterRole/cluster-admin                                                          6h8m
cluster-admin-binding                                  ClusterRole/cluster-admin                                                          2m23s
kubeadm:get-nodes                                      ClusterRole/kubeadm:get-nodes                                                      6h8m
kubeadm:kubelet-bootstrap                              ClusterRole/system:node-bootstrapper                                               6h8m
kubeadm:node-autoapprove-bootstrap                     ClusterRole/system:certificates.k8s.io:certificatesigningrequests:nodeclient       6h8m
kubeadm:node-autoapprove-certificate-rotation          ClusterRole/system:certificates.k8s.io:certificatesigningrequests:selfnodeclient   6h8m

In [28]:
kubectl get clusterrolebindings cluster-admin-binding  -o yaml

apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  creationTimestamp: "2022-01-18T15:31:19Z"
  name: cluster-admin-binding
  resourceVersion: "16471"
  uid: 5ccdeeee-81af-41cc-b7bb-ad959931d65d
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: cluster-admin
subjects:
- kind: ServiceAccount
  name: default
  namespace: default


In [29]:
kubectl get clusterrolebindings -o jsonpath='{.items[*].subjects[*].name}' | grep --color=always default

system:masters default system:bootstrappers:kubeadm:default-node-token system:bootstrappers:kubeadm:default-node-token system:bootstrappers:kubeadm:default-node-token system:nodes kube-proxy kubernetes-dashboard default storage-provisioner system:authenticated attachdetach-controller certificate-controller clusterrole-aggregation-controller cronjob-controller daemon-set-controller deployment-controller disruption-controller endpoint-controller endpointslice-controller endpointslicemirroring-controller ephemeral-volume-controller expand-controller generic-garbage-collector horizontal-pod-autoscaler job-controller namespace-controller node-controller persistent-volume-binder pod-garbage-collector pv-protection-controller pvc-protection-controller replicaset-controller replication-controller resourcequota-controller root-ca-cert-publisher route-controller service-account-controller service-controller statefulset-controller ttl-after-finished-controller ttl-controller coredns system:authen

In [30]:
kubectl get clusterrolebindings -o jsonpath='{range .items[*]}{.metadata.name}{":\t"}{.roleRef.name}{" --> "}{ range.subjects[*]}{.namespace}{":"}{.name}{end}{"\n"}{end}' | grep --color=always -E '$|default|cluster-admin'

cluster-admin:	cluster-admin --> :system:masters
cluster-admin-binding:	cluster-admin --> default:default
kubeadm:get-nodes:	kubeadm:get-nodes --> :system:bootstrappers:kubeadm:default-node-token
kubeadm:kubelet-bootstrap:	system:node-bootstrapper --> :system:bootstrappers:kubeadm:default-node-token
kubeadm:node-autoapprove-bootstrap:	system:certificates.k8s.io:certificatesigningrequests:nodeclient --> :system:bootstrappers:kubeadm:default-node-token
kubeadm:node-autoapprove-certificate-rotation:	system:certificates.k8s.io:certificatesigningrequests:selfnodeclient --> :system:nodes
kubeadm:node-proxier:	system:node-proxier --> kube-system:kube-proxy
kubernetes-dashboard:	cluster-admin --> kubernetes-dashboard:kubernetes-dashboard
minikube-rbac:	cluster-admin --> kube-system:default
storage-provisioner:	system:persistent-volume-provisioner --> kube-system:storage-provisioner
system:basic-user:	system:basic-user --> :system:authenticated
system:controller:attachdetach-controller:	system:

In [31]:
kubectl get rolebindings -A -o jsonpath='{range .items[*]}{.metadata.namespace}{":"}{.metadata.name}{":\t"}{.roleRef.name}{" --> "}{ range.subjects[*]}{.namespace}{":"}{.name}{end}{"\n"}{end}' | grep --color=always -E '$|default|cluster-admin'

kube-public:kubeadm:bootstrap-signer-clusterinfo:	kubeadm:bootstrap-signer-clusterinfo --> :system:anonymous
kube-public:system:controller:bootstrap-signer:	system:controller:bootstrap-signer --> kube-system:bootstrap-signer
kube-system:kube-proxy:	kube-proxy --> :system:bootstrappers:kubeadm:default-node-token
kube-system:kubeadm:kubelet-config-1.22:	kubeadm:kubelet-config-1.22 --> :system:nodes:system:bootstrappers:kubeadm:default-node-token
kube-system:kubeadm:nodes-kubeadm-config:	kubeadm:nodes-kubeadm-config --> :system:bootstrappers:kubeadm:default-node-token:system:nodes
kube-system:system::extension-apiserver-authentication-reader:	extension-apiserver-authentication-reader --> :system:kube-controller-manager:system:kube-scheduler
kube-system:system::leader-locking-kube-controller-manager:	system::leader-locking-kube-controller-manager --> :system:kube-controller-managerkube-system:kube-controller-manager
kube-system:system::leader-locking-kube-scheduler:	system::leader-locking-

[removing the serviceaccount token from the container](ServiceAccountToken.ipynb)

[Endless Discussion](https://github.com/kubernetes/kubernetes/issues/57601#issuecomment-353824159)

# Cleanup

In [34]:
kubectl delete clusterrolebinding cluster-admin-binding

Error from server (NotFound): clusterrolebindings.rbac.authorization.k8s.io "cluster-admin-binding" not found


: 1